In [335]:
def createdic(start,end):
    dic={}
    for i in range(int(start),int(end)):
        dic[i]='https://en.wikipedia.org/wiki/{}_Mosconi_Cup'.format(i)
    return(dic)

In [166]:
def createtables(url):
    import requests
    from lxml import html
    page = requests.get(url)
    tree = html.fromstring(page.text)
    tables = tree.xpath('//table')
    tables=[table for table in tables if(table[0].text_content().replace('\n','')=='Results')]
    return(tables)

In [320]:
def cdf(year,tables,df):
    import pandas as pd
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i,table in enumerate(tables):
        for j in range(len(table)-2):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    l.append(list(match.groups()))
            except:
                pass
    print(l)
    if df.empty:
        df=pd.DataFrame(l,columns=col)
        df['Year']=year
        df.index.rename('Match',inplace=True)
        df.reset_index(inplace=True)
        df.set_index(['Year','Match'],inplace=True)
    else:
        new = pd.DataFrame(l, columns = col)
        new['Year']=year
        new.index.rename('Match',inplace=True)
        new.reset_index(inplace=True)
        new.set_index(['Year','Match'],inplace=True)
        frames = (df,new)
        df = pd.concat(frames)
    return(df)

In [321]:
def singlemain():
    years,links = createdic(1998,2014)
    year=years[0]
    link=links[0]
    df = pd.DataFrame()    
    table = createtables(link)
    df = cdf(year,table,df)
    return(df)
s = singlemain()

[['Doubles', 'Mika Immonen Ralf Souquet', '7', '5', 'Johnny Archer Nick Varner'], ['Doubles', 'Oliver Ortmann Steve Knight', '7', '6', 'James Rempe Earl Strickland'], ['Doubles', 'Fabio Petroni Steve Davis', '5', '7', 'Kim Davenport Reed Pierce'], ['Doubles', 'Mika Immonen Ralf Souquet', '7', '3', 'James Rempe Earl Strickland'], ['Doubles', 'Steve Davis Fabio Petroni', '3', '7', 'Nick Varner Johnny Archer'], ['Doubles', 'Oliver Ortmann Steve Knight', '7', '5', 'Kim Davenport Reed Pierce'], ['Singles', 'Oliver Ortmann', '3', '7', 'Johnny Archer'], ['Singles', 'Steve Knight', '3', '7', 'Earl Strickland'], ['Singles', 'Mika Immonen', '4', '7', 'Reed Pierce'], ['Singles', 'Ralf Souquet', '7', '1', 'Nick Varner'], ['Doubles', 'Steve Knight Oliver Ortmann', '6', '7', 'Earl Strickland Kim Davenport'], ['Doubles', 'Ralf Souquet Mika Immonen', '5', '7', 'Johnny Archer Nick Varner'], ['Doubles', 'Steve Davis Fabio Petroni', '7', '2', 'Reed Pierce James Rempe'], ['Singles', 'Steve Davis', '2', '7

In [331]:
def newmain():
    dic = createdic(1998,2014)
    df = pd.DataFrame(columns = col)
    for year,link in dic.items():
        tables = createtables(link)
        df = cdf(year,tables,df)
    return(df)

In [334]:
attempt=newmain()
attempt

[['Doubles', 'Mika Immonen Ralf Souquet', '7', '5', 'Johnny Archer Nick Varner'], ['Doubles', 'Oliver Ortmann Steve Knight', '7', '6', 'James Rempe Earl Strickland'], ['Doubles', 'Fabio Petroni Steve Davis', '5', '7', 'Kim Davenport Reed Pierce'], ['Doubles', 'Mika Immonen Ralf Souquet', '7', '3', 'James Rempe Earl Strickland'], ['Doubles', 'Steve Davis Fabio Petroni', '3', '7', 'Nick Varner Johnny Archer'], ['Doubles', 'Oliver Ortmann Steve Knight', '7', '5', 'Kim Davenport Reed Pierce'], ['Singles', 'Oliver Ortmann', '3', '7', 'Johnny Archer'], ['Singles', 'Steve Knight', '3', '7', 'Earl Strickland'], ['Singles', 'Mika Immonen', '4', '7', 'Reed Pierce'], ['Singles', 'Ralf Souquet', '7', '1', 'Nick Varner'], ['Doubles', 'Steve Knight Oliver Ortmann', '6', '7', 'Earl Strickland Kim Davenport'], ['Doubles', 'Ralf Souquet Mika Immonen', '5', '7', 'Johnny Archer Nick Varner'], ['Doubles', 'Steve Davis Fabio Petroni', '7', '2', 'Reed Pierce James Rempe'], ['Singles', 'Steve Davis', '2', '7

[['Doubles', 'Thorsten Hohmann Niels Feijen', '5', '1', 'Johnny Archer Jeremy Jones'], ['Singles', 'Mika Immonen', '5', '2', 'Johnny Archer'], ['Doubles', 'Alex Lely Raj Hundal', '3', '5', 'Charlie Williams Shaun Putnam'], ['Singles', 'Niels Feijen', '3', '5', 'Charlie Williams'], ['Doubles', 'Mika Immonen Marcus Chamat', '4', '5', 'Earl Strickland Rodney Morris'], ['Doubles', 'Thorsten Hohmann Niels Feijen', '4', '5', 'Charlie Williams Shaun Putnam'], ['Singles', 'Raj Hundal', '5', '4', 'Shaun Putnam'], ['Doubles', 'Mika Immonen Marcus Chamat', '4', '5', 'Johnny Archer Jeremy Jones'], ['Singles', 'Thorsten Hohmann', '5', '4', 'Jeremy Jones'], ['Doubles', 'Alex Lely Raj Hundal', '2', '5', 'Earl Strickland Rodney Morris'], ['Doubles', 'Niels Feijen Alex Lely', '5', '3', 'Charlie Williams Shaun Putnam'], ['Singles', 'Alex Lely', '4', '5', 'Rodney Morris'], ['Doubles', 'Marcus Chamat Mika Immonen', '2', '5', 'Rodney Morris Earl Strickland'], ['Singles', 'Marcus Chamat', '2', '5', 'Earl St

[['Teams', 'Team Europe', '3', '5', 'Team USA'], ['Doubles', 'Chris Melling Nick van den Berg', '4', '5', 'Johnny Archer Brandon Shuff'], ['Singles', 'Niels Feijen', '5', '0', 'Mike Dechaine'], ['Doubles', 'Darren Appleton Nick Ekonomopoulos', '4', '5', 'Dennis Hatch Shane Van Boening'], ['Singles', 'Darren Appleton', '5', '2', 'Brandon Shuff'], ['Doubles', 'Darren Appleton Chris Melling', '4', '5', 'Johnny Archer Mike Dechaine'], ['Singles', 'Nick Ekonomopoulos', '5', '3', 'Shane Van Boening'], ['Doubles', 'Niels Feijen Nick van den Berg', '5', '2', 'Dennis Hatch Brandon Shuff'], ['Singles', 'Chris Melling', '5', '3', 'Dennis Hatch'], ['Doubles', 'Darren Appleton Niels Feijen', '4', '5', 'Shane Van Boening Johnny Archer'], ['Doubles', 'Nick van den Berg Nick Ekonomopoulos', '5', '4', 'Mike Dechaine Brandon Shuff'], ['Singles', 'Nick van den Berg', '4', '5', 'Johnny Archer'], ['Doubles', 'Nick Ekonomopoulos Niels Feijen', '5', '4', 'Dennis Hatch Mike Dechaine'], ['Singles', 'Chris Mell

Format                       European_player European_score  \
Year Match                                                                 
1998 0      Doubles             Mika Immonen Ralf Souquet              7   
     1      Doubles           Oliver Ortmann Steve Knight              7   
     2      Doubles             Fabio Petroni Steve Davis              5   
     3      Doubles             Mika Immonen Ralf Souquet              7   
     4      Doubles             Steve Davis Fabio Petroni              3   
     5      Doubles           Oliver Ortmann Steve Knight              7   
     6      Singles                        Oliver Ortmann              3   
     7      Singles                          Steve Knight              3   
     8      Singles                          Mika Immonen              4   
     9      Singles                          Ralf Souquet              7   
     10     Doubles           Steve Knight Oliver Ortmann              6   
     11     Doubles             Ralf Souquet Mika Immonen              5   
     12     Doubles             Steve Davis Fabio Petroni              7   
     13     Singles                           Steve Davis              2   
     14     Singles                         Fabio Petroni              5   
     15     Singles                          Mika Immonen              4   
     16     Singles                          Ralf Souquet              7   
     17     Singles                        Oliver Ortmann              4   
     18     Singles                          Steve Knight              7   
     19     Singles                           Steve Davis              7   
     20     Singles                         Fabio Petroni              5   
     21     Singles                          Ralf Souquet              4   
1999 0      Doubles             Mika Immonen Ralf Souquet              2   
     1      Doubles              Oliver Ortmann Alex Lely              2   
     2      Doubles              Steve Davis Steve Knight              4   
     3      Doubles           Ralf Souquet Oliver Ortmann              5   
     4      Doubles             Steve Knight Mika Immonen              5   
     5      Doubles                 Steve Davis Alex Lely              5   
     6      Singles                             Alex Lely              4   
     7      Singles                          Ralf Souquet              5   
...             ...                                   ...            ...   
2012 3      Doubles    Darren Appleton Nick Ekonomopoulos              4   
     4      Singles                       Darren Appleton              5   
     5      Doubles         Darren Appleton Chris Melling              4   
     6      Singles                    Nick Ekonomopoulos              5   
     7      Doubles        Niels Feijen Nick van den Berg              5   
     8      Singles                         Chris Melling              5   
     9      Doubles          Darren Appleton Niels Feijen              4   
     10     Doubles  Nick van den Berg Nick Ekonomopoulos              5   
     11     Singles                     Nick van den Berg              4   
     12     Doubles       Nick Ekonomopoulos Niels Feijen              5   
     13     Singles                         Chris Melling              5   
     14     Singles                       Darren Appleton              5   
     15     Singles                    Nick Ekonomopoulos              3   
     16     Singles                          Niels Feijen              1   
     17     Singles                       Darren Appleton              1   
     18     Singles                         Chris Melling              5   
     19     Singles                     Nick van den Berg              5   
2013 0        Teams                           Team Europe              6   
     1      Doubles            Darren Appleton Karl Boyes              6   
     2      Singles                          Mika Immonen    

In [3]: tuples
Out[3]: 
[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
need tuples to be (year, match_no)

In [170]:
df = newmain()

Missing 2014 data. Can get it from: http://www.matchroompool.com/mosconi-cup/mosconi-cup-xxi/#fixtures
compare stats here:
http://forums.azbilliards.com/showthread.php?t=439775&page=4

In [176]:
import re
import pandas as pd
l=[]
col=['Format','European_player','European_score','American_score','American_player']

with open('manual_scrape_cleaned_2014.txt','r') as rf:
    for line in rf:
        match = re.match(r"(\w+)\s(.+)\s([0-9])\s?[-]\s?([0-9])\s?(.+)", line)
        if match:
            l.append(list(match.groups()))
        else:
            print('fail')
df2014
# df2014=pd.DataFrame(l,columns=col)
# df2014['Europe_won']=df.European_score>df.American_score
# df2014

,Format,European_player,European_score,American_score,American_player
0,Teams,Team Europe,5,1,Team USA
1,Doubles,Darren Appleton & Karl Boyes,3,5,Corey Deuel & Justin Hall
2,Singles,Mark Gray,3,5,John Schmidt
3,Doubles,Niels Feijen & Nikos Ekonomopoulos,5,1,Shane Van Boening & Justin Bergman
4,Singles,Darren Appleton,5,3,Shane Van Boening
5,Doubles,Niels Feijen & Mark Gray,2,5,Justin Hall & Justin Bergman
6,Singles,Karl Boyes,5,2,Corey Deuel
7,Doubles,Darren Appleton & Nikos Ekonomopoulos,5,2,Shane Van Boening & John Schmidt
8,Singles,Niels Feijen,5,3,Justin Bergman
9,Doubles,Darren Appleton & Mark Gray,4,5,Shane Van Boening & Corey Deuel


In [177]:
frames=[df,df2014]
df=pd.concat(frames)

2015 Data is backwards, need to fix the order (european player should be first)

In [191]:
def createdf_bw(tables):
    import re
    l=[]
    col=['Format','European_player','European_score','American_score','American_player']

    for i in range(len(tables)+1):
        for j in range(1,6):
            try:
                element=tables[i][j].text_content().replace("\n", " ")
                match = re.match(r"\s?(\w+)\s(.+)\s([0-9])[–]([0-9])\s?\w+\s?(.+)\s", element)
                if match:
                    lst=[match.group(1), match.group(5), match.group(4), match.group(3), match.group(2)]
                    l.append(lst)
            except:
                pass
    print(l)
    df=pd.DataFrame(l,columns=col)
    return(df)

In [192]:
def main_bw():
    from lxml import html
    import requests
    import pandas as pd
    df = pd.DataFrame()
    links = createlinks('2015','2016')
    for url in links:
        tables = createtables(url)
        tl = tablelengths(tables)
        newdf = createdf_bw(tables) 
        df=df.append(newdf)
    return(df)

In [193]:
df2015= main_bw()

[['Teams', 'Team Europe', '4', '5', 'Team USA'], ['Doubles', 'Karl Boyes Nick van den Berg', '5', '4', 'Corey Deuel Mike Dechaine'], ['Singles', 'Niels Feijen', '5', '2', 'Justin Bergman'], ['Doubles', 'Darren Appleton Albin Ouschan', '1', '5', 'Shane Van Boening Skyler Woodward'], ['Singles', 'Darren Appleton', '5', '1', 'Shane Van Boening'], ['Doubles', 'Niels Feijen Nick van den Berg', '3', '5', 'Justin Bergman Skyler Woodward'], ['Singles', 'Albin Ouschan', '5', '3', 'Mike Dechaine'], ['Doubles', 'Darren Appleton Karl Boyes', '5', '2', 'Shane Van Boening Corey Deuel'], ['Singles', 'Karl Boyes', '5', '3', 'Skyler Woodward'], ['Doubles', 'Darren Appleton Nick van den Berg', '4', '5', 'Shane Van Boening Mike Dechaine'], ['Doubles', 'Niels Feijen Albin Ouschan', '5', '1', 'Justin Bergman Corey Deuel'], ['Singles', 'Nick van den Berg', '4', '5', 'Corey Deuel'], ['Doubles', 'Karl Boyes Albin Ouschan', '5', '2', 'Mike Dechaine Skyler Woodward'], ['Singles', 'Niels Feijen', '5', '4', 'Shan

In [182]:
frames=[df,df2015]
df=pd.concat(frames)

In [185]:
pkl = 'new_inc_2015_fixed_again'
pd.to_pickle(df,pkl)

Next steps: Rescrape the data. 

1. Need to include the year as a multiindex and concatenate the frames per year.(done)
1. Need to reconfigure the number of rows to possibly extend beyond 5. Have to dynamically learn the number of rows in the    given table and then use that number instead of hard coding it. (done)
3. Need to collect the 'team' data each year. I.e. what players are on each team so that I can do analysis on the 'team' matches and include that in the final presentations.
4. Scrape location or just see if it changes every year

1. Reconfigure number of rows dynamically.